# Домашнее задание № 3. Исправление опечаток

# 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

In [2]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [3]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [4]:
vocab = Counter(re.findall('\w+', corpus.lower()))

In [5]:
!pip install textdistance

In [6]:
import textdistance

In [7]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest_with_freq = [(word, dist, P(text, N)) for word, dist in closest]

    return sorted(closest_with_freq, key=lambda x: (x[1], x[2]), reverse=True)


def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]
    
    return[(id2word[top], similarities[top]) for top in topn]

    

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [8]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [9]:
get_closest_hybrid_match('грибый', X, vec)

[('грибы', 0.8333333333333334, 0.0),
 ('грубый', 0.8333333333333334, 0.0),
 ('игривый', 0.7142857142857143, 0.0),
 ('грибным', 0.7142857142857143, 0.0),
 ('грибные', 0.7142857142857143, 0.0)]

In [10]:
get_closest_hybrid_match('чесмный', X, vec)

[('честный', 0.8571428571428572, 0.0),
 ('темный', 0.7142857142857143, 0.0),
 ('тесный', 0.7142857142857143, 0.0),
 ('бесшумный', 0.6666666666666667, 0.0),
 ('месячный', 0.625, 0.0)]

# 2. Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово

Оцените качество полученного алгоритма теми же тремя метриками.

In [11]:
correct_dic = list(vocab.keys())

In [12]:
deleted_dic = {}
for word in correct_dic:
    for i in range(len(word)):
        wwd = word[0:i] + word[i+1:]
        if wwd in deleted_dic.keys():
            deleted_dic[wwd] = deleted_dic[wwd] + [word]
        else:
            deleted_dic[wwd] = [word]

In [13]:
def fix_errors(text, deleted_dic):
    similar = set()
    for i in range(len(text)):
        if text[0:i] + text[i+1:] in deleted_dic.keys():
            for word in deleted_dic[text[0:i]  + text[i+1:]]:
                similar.add(word)
    similar_freq = [(word, P(word, N)) for word in similar]
    if len(similar_freq) == 0:
        return ' '
    return sorted(similar_freq, key=lambda x: x[1], reverse=True)

In [14]:
fix_errors('грибый', deleted_dic)

[('грубый', 1.745783302901744e-06)]

In [15]:
fix_errors('чесмный', deleted_dic)

[('честный', 1.5518073803571057e-06)]

In [16]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

mistakes = []

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], fix_errors(pair[1], deleted_dic)[0][0])
            cashed[pair[1]] = pred
            
        else:
            pred = pair[1]
            
        if pred == pair[0]:
            correct += 1
            
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
                
    if not i % 100:
        print(i)
    
    

0
100
200
300
400
500
600
700
800
900


In [17]:
print(correct / total)
print(mistaken_fixed / total_mistaken)
print(correct_broken / total_correct)

0.8185092546273136
0.20031055900621117
0.09004249454461927
